<a href="https://colab.research.google.com/github/iskra3138/UDA/blob/main/Office31_D_AW_by_LGAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Mar 24 02:03:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading dataset

### office31 dataset

In [ ]:
!gdown  https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE

Downloading...
From: https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE
To: /content/domain_adaptation_images.tar.gz
77.2MB [00:01, 76.0MB/s]


In [ ]:
!tar -xvzf domain_adaptation_images.tar.gz

amazon/images/back_pack/frame_0001.jpg
amazon/images/back_pack/frame_0002.jpg
amazon/images/back_pack/frame_0003.jpg
amazon/images/back_pack/frame_0004.jpg
amazon/images/back_pack/frame_0005.jpg
amazon/images/back_pack/frame_0006.jpg
amazon/images/back_pack/frame_0007.jpg
amazon/images/back_pack/frame_0008.jpg
amazon/images/back_pack/frame_0009.jpg
amazon/images/back_pack/frame_0010.jpg
amazon/images/back_pack/frame_0011.jpg
amazon/images/back_pack/frame_0012.jpg
amazon/images/back_pack/frame_0013.jpg
amazon/images/back_pack/frame_0014.jpg
amazon/images/back_pack/frame_0015.jpg
amazon/images/back_pack/frame_0016.jpg
amazon/images/back_pack/frame_0017.jpg
amazon/images/back_pack/frame_0018.jpg
amazon/images/back_pack/frame_0019.jpg
amazon/images/back_pack/frame_0020.jpg
amazon/images/back_pack/frame_0021.jpg
amazon/images/back_pack/frame_0022.jpg
amazon/images/back_pack/frame_0023.jpg
amazon/images/back_pack/frame_0024.jpg
amazon/images/back_pack/frame_0025.jpg
amazon/images/back_pack/f

In [ ]:
ll

total 75396
drwxr-xr-x 3 root     4096 Mar 24 02:03 amazon/
-rw-r--r-- 1 root 77187046 Mar 24 02:03 domain_adaptation_images.tar.gz
drwxr-xr-x 3 root     4096 Mar 24 02:03 dslr/
drwxr-xr-x 1 root     4096 Mar 18 13:36 sample_data/
drwxr-xr-x 3 root     4096 Mar 24 02:03 webcam/


# HyperParameter

In [ ]:
NUM_WORKERS = 0
IMAGE_SIZE = (224,224)
BATCHSIZE = 32
folder = './data/'
names = ['amazon', 'dslr', 'webcam']
CLASS_NUM = 31
SOURCE = names[0]
TARGET = names[1]
TRTE = 'val'
DSET = 'office' 
#NET = 'resnet50' 
MAX_EPOCH=20

PRETRAINED = 'PyTorch' # 'PyTorch' or 'TF'
OPTIMIZER = 'adam' #'sgd' or 'adam'
LOSS = 'default' # 'shot' or 'default'

assert PRETRAINED in ["PyTorch", "TF"], "PRETRAINED should be 'PyTorch' or 'TF'"
assert OPTIMIZER in ["sgd", "adam"], "OPTIMIZER should be 'sgd' or 'adam'"
assert LOSS in ["shot", "default"], "OPTIMIZER should be 'shot' or 'default'"

NAME_SRC = SOURCE[0].upper()

S_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(SOURCE)
TEST_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(TARGET)

# Data Pipeline

In [ ]:
# Making files about paths of images
# ref: https://github.com/tim-learn/SHOT/issues/2 

import os, glob 
data_path = './data/office'
if not os.path.exists(data_path) :
  os.makedirs(data_path)

datasets = ['amazon', 'dslr', 'webcam']
classes = os.listdir('./amazon/images/')

for dataset in datasets:
  img_paths_file = open(os.path.join(data_path,'{}_list.txt'.format(dataset)), 'w')
  for i, cls in enumerate(classes): 
    for filename in glob.glob(os.path.join(dataset,'images',cls,'*.jpg')):
      img_paths_file.write("{} {}\n".format(filename,i))
  img_paths_file.close()

In [ ]:
import os
import tqdm
import torchvision
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from torch.utils.data import Dataset
#from data_list import ImageList
import random

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

def make_dataset(image_list, labels):
    if labels:
      len_ = len(image_list)
      images = [(image_list[i].strip(), labels[i, :]) for i in range(len_)]
    else:
      if len(image_list[0].split()) > 2:
        images = [(val.split()[0], np.array([int(la) for la in val.split()[1:]])) for val in image_list]
      else:
        images = [(val.split()[0], int(val.split()[1])) for val in image_list]
    return images

def rgb_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
            
class ImageList(Dataset):
    def __init__(self, image_list, labels=None, transform=None, target_transform=None, mode='RGB'):
        imgs = make_dataset(image_list, labels)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(IMG_EXTENSIONS)))

        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        if mode == 'RGB':
            self.loader = rgb_loader
        elif mode == 'L':
            self.loader = l_loader

    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.imgs)

def image_train(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.RandomCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

def image_test(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        normalize
    ])
  
def data_load(): 
    ## prepare data
    dsets = {}
    dset_loaders = {}
    train_bs = BATCHSIZE
    txt_src = open(S_DSET_PATH).readlines()
    txt_test = open(TEST_DSET_PATH).readlines()

    if TRTE == "val":
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        # print(dsize, tr_size, dsize - tr_size)
        tr_txt, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
    else:
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        _, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
        tr_txt = txt_src

    dsets["source_tr"] = ImageList(tr_txt, transform=image_train())
    dset_loaders["source_tr"] = DataLoader(dsets["source_tr"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["source_te"] = ImageList(te_txt, transform=image_test())
    dset_loaders["source_te"] = DataLoader(dsets["source_te"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["test"] = ImageList(txt_test, transform=image_test())
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=train_bs*2, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)

    return dset_loaders


In [ ]:
dset_loaders = data_load()

acc_init = 0
max_iter = MAX_EPOCH * len(dset_loaders["source_tr"])
interval_iter = max_iter // 10
iter_num = 0

# Training by Tensorflow2

In [ ]:
# Install tensoflow-addons for using WeightNormalization
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 706kB 18.6MB/s 


In [ ]:
# Install pytorch2keras for converting PyTorch pretrained model to Tensorflow model.
!pip install pytorch2keras

     |████████████████████████████████| 14.5MB 237kB/s 
  Created wheel for pytorch2keras: filename=pytorch2keras-0.2.4-cp37-none-any.whl size=29664 sha256=6ec7b0b5fc3cf2217dc4abed6f3f0b1d19137ff9a215bbde532e6d3fcc407700
  Stored in directory: /root/.cache/pip/wheels/36/8b/2e/e2b6ae7a78ad4661e6156700bf96816309013f89f7a21f82d7
  Created wheel for onnx2keras: filename=onnx2keras-0.0.24-cp37-none-any.whl size=24582 sha256=ec78ed2007e203ffdb278cc3e8879460e192c64a7ff32261c114319ceede067d
  Stored in directory: /root/.cache/pip/wheels/1d/d2/0a/8310ecf5f80355546fadda0fb4c611c8b54a63fce9af555155
Successfully built pytorch2keras onnx2keras


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from pytorch2keras import pytorch_to_keras
from torch.autograd import Variable

print("Tensorflow version " + tf.__version__)
CLASSIFIER_TYPE='wn'

In [ ]:
# Defining CustomDecay Class same to learning rate decay of SHOT Paper
##### decay = (1 + gamma * iter_num / max_iter) ** (-power)

from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

class CustomDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(
      self,
      initial_learning_rate,
      max_iter,
      gamma=10,
      power=0.75,
      name=None):

    super(CustomDecay, self).__init__()
    self.initial_learning_rate = initial_learning_rate
    self.max_iter = max_iter
    self.gamma = gamma
    self.power = power
    self.name = name

  def __call__(self, step):
    with ops.name_scope_v2(self.name or "lrDecay") as name:
      initial_learning_rate = ops.convert_to_tensor(self.initial_learning_rate, name="initial_learning_rate")
      dtype = initial_learning_rate.dtype

      max_iter = math_ops.cast(self.max_iter, dtype)
      gamma = math_ops.cast(self.gamma, dtype)
      power = math_ops.cast(self.power, dtype)

      global_step_recomp = math_ops.cast(step, dtype)
      p = global_step_recomp / max_iter
      
      decay = math_ops.pow(math_ops.add(1.0,math_ops.multiply(gamma, p)),math_ops.multiply(-1.0,power))

      return math_ops.multiply(
          initial_learning_rate, decay, name=name)

  def get_config(self):
    return {
        "initial_learning_rate": self.initial_learning_rate,
        "max_iter": self.max_iter,
        "gamma": self.gamma,
        "power": self.power,
        "name": self.name
    }

In [ ]:
# adding L2 regularizer in the pretrained model. 
## (https://sthalles.github.io/keras-regularizer/)

import os
import tempfile

def add_regularization(model, regularizer=tf.keras.regularizers.l2(0.001)):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
      print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
      return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
              setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

In [ ]:
def create_model():
  if PRETRAINED == 'PyTorch' :
    pretrained_model = torchvision.models.resnet50(pretrained=True)
    input_np = np.random.uniform(0, 1, (1, 3, *IMAGE_SIZE))
    input_var = Variable(torch.FloatTensor(input_np))
    pretrained_model = pytorch_to_keras(pretrained_model, input_var, (3, *IMAGE_SIZE,), verbose=True, change_ordering=True)
    x = pretrained_model.layers[137].output #right before global average pooling
  else :
    pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    x = pretrained_model.output

  pretrained_model.trainable = True    

  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(256, 
                            kernel_initializer='glorot_normal', 
                            bias_initializer='zeros',
                            name='feature') (x)  
  x = tf.keras.layers.BatchNormalization(name='feature_bn')(x)
  x = tf.keras.layers.ReLU()(x)

  if CLASSIFIER_TYPE=='wn' :
    predictions = tfa.layers.WeightNormalization(tf.keras.layers.Dense(CLASS_NUM, 
                                                                      kernel_initializer='glorot_normal', 
                                                                      bias_initializer='zeros', 
                                                                      name='prediction'), data_init=False) (x)
  else :
    predictions = tf.keras.layers.Dense(CLASS_NUM, 
                                        kernel_initializer='glorot_normal', 
                                        bias_initializer='zeros', 
                                        name='prediction') (x)

  model = tf.keras.Model(inputs=pretrained_model.input, outputs=predictions)

  return model

In [ ]:
model = create_model()

In [ ]:
model = add_regularization(model)
#model.summary()

In [ ]:
lr_schedule1 = CustomDecay(
    initial_learning_rate=0.00001,
    max_iter = max_iter,
    gamma=10,
    power=0.75)

lr_schedule2 = CustomDecay(
    initial_learning_rate=0.001,
    max_iter = max_iter,
    gamma=10,
    power=0.75)

if OPTIMIZER == 'sgd' :
  optimizer1 = tf.keras.optimizers.SGD(learning_rate=lr_schedule1, momentum=0.9, nesterov=True)
  optimizer2 = tf.keras.optimizers.SGD(learning_rate=lr_schedule2, momentum=0.9, nesterov=True)
else:
  optimizer1 = tf.keras.optimizers.Adam(learning_rate=lr_schedule1)
  optimizer2 = tf.keras.optimizers.Adam(learning_rate=lr_schedule2)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [ ]:
variable_list1=[]
variable_list2=[]
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

@tf.function
def train_step(inputs, targets): 
  model.trainable = True 
  if PRETRAINED == 'PyTorch' :
    for var in model.trainable_variables:
      if 'feature' in var.name :
        variable_list2.append(var)
      elif 'weight_normalization' in var.name :
        variable_list2.append(var)
      else :
        variable_list1.append(var)

  else :
    for var in model.trainable_variables:
      if 'conv' in var.name :
        variable_list1.append(var)
      else :
        variable_list2.append(var)
  
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    logits = model(inputs, training=True)
    targets = tf.one_hot(targets, CLASS_NUM) # one-hot encoding
    targets = (1 - 0.1) * targets + 0.1 / CLASS_NUM
    if LOSS == 'shot' :
      log_probs = tf.nn.log_softmax(logits, name='preds')
      loss = tf.reduce_sum((- targets * log_probs), axis=1)
    else :
      loss = loss_object(targets, logits)

  gradients = tape.gradient(loss, variable_list1 + variable_list2)
  grads1 = gradients[:len(variable_list1)]
  grads2 = gradients[len(variable_list1):]

  train_op1 = optimizer1.apply_gradients(zip(grads1, variable_list1))
  train_op2 = optimizer2.apply_gradients(zip(grads2, variable_list2))

  train_op = tf.group(train_op1, train_op2)

  train_loss(loss)
  train_accuracy(targets, logits)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  model.trainable = False
  logits = model(images, training=False)
  labels = tf.one_hot(labels, CLASS_NUM)
  t_loss = tf.nn.softmax_cross_entropy_with_logits(labels, logits, axis=-1)

  test_loss(t_loss)
  test_accuracy(labels, logits)

In [ ]:
for epoch in range(MAX_EPOCH):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  iter_source = iter(dset_loaders["source_tr"])
  for i, _ in enumerate(dset_loaders["source_tr"]):
    inputs_source, labels_source = iter_source.next()
    inputs_source = inputs_source.permute(0, 2, 3, 1)
    train_step(inputs_source.numpy(), labels_source.numpy())

  if (epoch+1) % 10 == 0 :
    iter_source = iter(dset_loaders["test"])
    for i, _ in enumerate(dset_loaders["test"]):
      inputs_source, labels_source = iter_source.next()
      inputs_source = inputs_source.permute(0, 2, 3, 1)
      test_step(inputs_source.numpy(), labels_source.numpy())

    lr1 = optimizer1._decayed_lr(tf.float32)
    lr2 = optimizer2._decayed_lr(tf.float32)
    print(
      f'Epoch {epoch + 1}, '
      f'lr1: {lr1:.5f}, '
      f'lr2: {lr2:.5f}, '
      f'Loss: {train_loss.result():.2f}, '  
      f'Accuracy: {train_accuracy.result() * 100:.2f}, '  
      f'Test Loss: {test_loss.result():.2f}, '
      f'Test Accuracy: {test_accuracy.result() * 100:.2f}'
    )

In [ ]:
for i in range(len(names)):
    if names[i] == SOURCE:
        continue
    #test_dset_path = folder + args.dset + '/' + names[i] + '_list.txt'
    test_loss.reset_states()
    test_accuracy.reset_states()

    test_dset_path = folder + DSET + '/' + '{}_list.txt'.format(names[i])

    dsets = {}
    dset_loaders = {}
    txt_test = open(test_dset_path).readlines()
    dsets["test"] = ImageList(txt_test, transform=image_test())
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=BATCHSIZE*2, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)

    iter_source = iter(dset_loaders["test"])
    for _ in tqdm.tqdm(dset_loaders["test"]):
      inputs_source, labels_source = iter_source.next()
      inputs_source = inputs_source.permute(0, 2, 3, 1)
      test_step(inputs_source.numpy(), labels_source.numpy())

    print(
      f'DataSet: {names[i]}, '
      f'Test Loss: {test_loss.result():.2f}, '
      f'Test Accuracy: {test_accuracy.result() * 100:.2f}'
    )